<a href="https://colab.research.google.com/github/KawaiiZT/IPA/blob/main/Jinja2_PNE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Jinja2: Getting Started

Jinja2 is a template language used in Python.

Jinja2 is used to generate documents based on one or more templates.

Examples of use:
- templates for generating HTML pages
- templates for generating configuration files in Unix/Linux
- templates for generating network device configuration files

The main idea of Jinja is to separate data and template. This allows you to use the same template but not the same data. In the simplest case, template is simply a text file that specifies locations of Jinja variables.

## Preparation

In [1]:
!pip install jinja2

Upload data_files.zip and templates.zip

In [3]:
!unzip data_files.zip
!unzip templates.zip

Archive:  data_files.zip
replace __MACOSX/._data_files? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  templates.zip
replace __MACOSX/._templates? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# Example of using Jinja

Template templates/router_template.txt is a plain text file: Copy to a file and upload to Google colab.

```
hostname {{name}}
!
interface Loopback10
 description MPLS loopback
 ip address 10.10.{{id}}.1 255.255.255.255
 !
interface GigabitEthernet0/0
 description WAN to {{name}} sw1 G0/1
!
interface GigabitEthernet0/0.1{{id}}1
 description MPLS to {{to_name}}
 encapsulation dot1Q 1{{id}}1
 ip address 10.{{id}}.1.2 255.255.255.252
 ip ospf network point-to-point
 ip ospf hello-interval 1
 ip ospf cost 10
!
interface GigabitEthernet0/1
 description LAN {{name}} to sw1 G0/2 !
interface GigabitEthernet0/1.{{IT}}
 description PW IT {{name}} - {{to_name}}
 encapsulation dot1Q {{IT}}
 xconnect 10.10.{{to_id}}.1 {{id}}11 encapsulation mpls
 backup peer 10.10.{{to_id}}.2 {{id}}21
  backup delay 1 1
!
interface GigabitEthernet0/1.{{BS}}
 description PW BS {{name}} - {{to_name}}
 encapsulation dot1Q {{BS}}
 xconnect 10.10.{{to_id}}.1 {{to_id}}{{id}}11 encapsulation mpls
  backup peer 10.10.{{to_id}}.2 {{to_id}}{{id}}21
  backup delay 1 1
!
router ospf 10
 router-id 10.10.{{id}}.1
 auto-cost reference-bandwidth 10000
 network 10.0.0.0 0.255.255.255 area 0
```

In Jinja, variables are written in double curly braces.

When script is executed, these variables are replaced with desired values.

Data file routers_info.yml: Copy to a file and upload to Google colab.

data_files/routers_info.yml

```
- id: 11
  name: Liverpool
  to_name: LONDON
  IT: 791
  BS: 1550
  to_id: 1

- id: 12
  name: Bristol
  to_name: LONDON
  IT: 793
  BS: 1510
  to_id: 1

- id: 14
  name: Coventry
  to_name: Manchester
  IT: 892
  BS: 1650
  to_id: 2
```


This template can be used to generate configuration of different devices by substituting other sets of variables.

In [ ]:
from jinja2 import Environment, FileSystemLoader
import yaml

env = Environment(loader=FileSystemLoader('templates'))
template = env.get_template('router_template.txt')

with open('data_files/routers_info.yml') as f:
    routers = yaml.safe_load(f)

for router in routers:
    r1_conf = router['name'] + '_r1.txt'
    with open(r1_conf, 'w') as f:
        f.write(template.render(router))

File router_config_generator.py imports from jinja2 module:

FileSystemLoader - a loader that allows working with a file system path to directory where templates are located is specified here in this case template is in templates directory

Environment - a class for describing environment parameters. In this case only loader is specified, but you can specify how to process a template
Note that template is now in templates directory.

# Jinja2 template syntax

So far, only variable substitution has been used in Jinja2 template examples. This is the simplest and most understandable example of using templates. Syntax of Jinja templates is not limited to this.

In Jinja2 templates you can use :

- variables
- conditions (if/else)
- loops (for)
- filters - special built-in methods that allow to convert variables
- tests - are used to check whether a variable matches a condition

In addition, Jinja supports inheritance between templates and also allows adding the contents of one template to another.

for.txt

```
hostname {{ name }}

interface Loopback0
 ip address 10.0.0.{{ id }} 255.255.255.255

{% for vlan, name in vlans.items() %}
vlan {{ vlan }}
 name {{ name }}
{% endfor %}

router ospf 1
 router-id 10.0.0.{{ id }}
 auto-cost reference-bandwidth 10000
{% for networks in ospf %}
 network {{ networks.network }} area {{ networks.area }}
{% endfor %}
```

for.yml

```
id: 3
name: R3
vlans:
  10: Marketing
  20: Voice
  30: Management
ospf:
  - network: 10.0.1.0 0.0.0.255
    area: 0
  - network: 10.0.2.0 0.0.0.255
    area: 2
  - network: 10.1.1.0 0.0.0.255
    area: 0
```

Script: cfg_gen.py

In [ ]:
from jinja2 import Environment, FileSystemLoader
import yaml

# python cfg_gen.py
# templates/for.txt
# data_files/for.yml
# template_dir, template_file = os.path.split(sys.argv[1])
# vars_file = sys.argv[2]

template_dir = "templates"
template_file = "templates/for.txt"
vars_file = 'data_files/for.yml'

env = Environment(
    loader=FileSystemLoader(template_dir),
    trim_blocks=True,
    lstrip_blocks=True)
template = env.get_template(template_file)

with open(vars_file) as f:
    vars_dict = yaml.safe_load(f)

print(template.render(vars_dict))

hostname R3

interface Loopback0
 ip address 10.0.0.3 255.255.255.255

vlan 10
 name Marketing
vlan 20
 name Voice
vlan 30
 name Management

router ospf 1
 router-id 10.0.0.3
 auto-cost reference-bandwidth 10000
 network 10.0.1.0 0.0.0.255 area 0
 network 10.0.2.0 0.0.0.255 area 2
 network 10.1.1.0 0.0.0.255 area 0



## Control of whitespace symbols


### trim_blocks, lstrip_blocks

Parameter trim_blocks removes the first empty line after block if its value is True (default False).

Effect of using the flag is showed on example templates/env_flags.txt:



```
router bgp {{ bgp.local_as }}
 {% for ibgp in bgp.ibgp_neighbors %}
 neighbor {{ ibgp }} remote-as {{ bgp.local_as }}
 neighbor {{ ibgp }} update-source {{ bgp.loopback }}
 {% endfor %}
 ```


If cfg_gen.py script starts without trim_blocks, lstrip_blocks:

In [ ]:
from jinja2 import Environment, FileSystemLoader
import yaml

template_dir = "templates"
template_file = "env_flags.txt"
vars_file = 'data_files/router.yml'

env = Environment(
    loader=FileSystemLoader(template_dir))
template = env.get_template(template_file)

with open(vars_file) as f:
    vars_dict = yaml.safe_load(f)

print(template.render(vars_dict))

router bgp 100
 
 neighbor 10.0.0.2 remote-as 100
 neighbor 10.0.0.2 update-source lo100
 
 neighbor 10.0.0.3 remote-as 100
 neighbor 10.0.0.3 update-source lo100
 


new lines occur because of for block.

{% for ibgp in bgp.ibgp_neighbors %}

By default, the same behavior will be with any other Jinja blocks.



When trim_blocks flag is added:

```
env = Environment(loader=FileSystemLoader(TEMPLATE_DIR),
                  trim_blocks=True)
```

In [ ]:
from jinja2 import Environment, FileSystemLoader
import yaml

template_dir = "templates"
template_file = "env_flags.txt"
vars_file = 'data_files/router.yml'

env = Environment(
    loader=FileSystemLoader(template_dir),
    trim_blocks=True)
template = env.get_template(template_file)

with open(vars_file) as f:
    vars_dict = yaml.safe_load(f)

print(template.render(vars_dict))

router bgp 100
  neighbor 10.0.0.2 remote-as 100
 neighbor 10.0.0.2 update-source lo100
  neighbor 10.0.0.3 remote-as 100
 neighbor 10.0.0.3 update-source lo100
 


Empty lines after block were removed.

In front of neighbor ... remote-as lines two spaces appeared. This is because there is a space in front of for block. Once lstrip_blocks has been disabled, spaces and tabs in front of the block are added to the first line of block.

This does not affect the next lines. Therefore, lines with neighbor ... update-source are displayed with one space.

Parameter lstrip_blocks controls whether spaces and tabs will be removed from the beginning of line to the beginning of block (until opening curly bracket).

If add lstrip_blocks=True:

```
env = Environment(loader=FileSystemLoader(TEMPLATE_DIR),
                  trim_blocks=True, lstrip_blocks=True)
```

In [ ]:
from jinja2 import Environment, FileSystemLoader
import yaml

template_dir = "templates"
template_file = "env_flags.txt"
vars_file = 'data_files/router.yml'

env = Environment(
    loader=FileSystemLoader(template_dir),
    trim_blocks=True,
    lstrip_blocks=True)
template = env.get_template(template_file)

with open(vars_file) as f:
    vars_dict = yaml.safe_load(f)

print(template.render(vars_dict))

router bgp 100
 neighbor 10.0.0.2 remote-as 100
 neighbor 10.0.0.2 update-source lo100
 neighbor 10.0.0.3 remote-as 100
 neighbor 10.0.0.3 update-source lo100



### Disabling lstrip_blocks for block

Sometimes you need to disable lstrip_blocks in block.

For example, if lstrip_blocks is set to True in an environment, but must be disabled for the second block in template (templates/env_flags2-1.txt file):

```
router bgp {{ bgp.local_as }}
 {% for ibgp in bgp.ibgp_neighbors %}
 neighbor {{ ibgp }} remote-as {{ bgp.local_as }}
 neighbor {{ ibgp }} update-source {{ bgp.loopback }}
 {% endfor %}

router bgp {{ bgp.local_as }}
 {%+ for ibgp in bgp.ibgp_neighbors %}
 neighbor {{ ibgp }} remote-as {{ bgp.local_as }}
 neighbor {{ ibgp }} update-source {{ bgp.loopback }}
 {% endfor %}
 ```

In [ ]:
from jinja2 import Environment, FileSystemLoader
import yaml

template_dir = "templates"
template_file = "env_flags2-1.txt"
vars_file = 'data_files/router.yml'

env = Environment(
    loader=FileSystemLoader(template_dir),
    trim_blocks=True,
    lstrip_blocks=True)
template = env.get_template(template_file)

with open(vars_file) as f:
    vars_dict = yaml.safe_load(f)

print(template.render(vars_dict))

router bgp 100
 neighbor 10.0.0.2 remote-as 100
 neighbor 10.0.0.2 update-source lo100
 neighbor 10.0.0.3 remote-as 100
 neighbor 10.0.0.3 update-source lo100

router bgp 100
  neighbor 10.0.0.2 remote-as 100
 neighbor 10.0.0.2 update-source lo100
 neighbor 10.0.0.3 remote-as 100
 neighbor 10.0.0.3 update-source lo100



Plus sign after percent sign disables lstrip_blocks for the block, in this case, only in the beginning.

If done this way (plus is added in the end block expression): - env_flags2-2.txt

```
router bgp {{ bgp.local_as }}
 {% for ibgp in bgp.ibgp_neighbors %}
 neighbor {{ ibgp }} remote-as {{ bgp.local_as }}
 neighbor {{ ibgp }} update-source {{ bgp.loopback }}
 {% endfor %}

router bgp {{ bgp.local_as }}
 {%+ for ibgp in bgp.ibgp_neighbors %}
 neighbor {{ ibgp }} remote-as {{ bgp.local_as }}
 neighbor {{ ibgp }} update-source {{ bgp.loopback }}
 {%+ endfor %}
 ```

In [ ]:
from jinja2 import Environment, FileSystemLoader
import yaml

template_dir = "templates"
template_file = "env_flags2-2.txt"
vars_file = 'data_files/router.yml'

env = Environment(
    loader=FileSystemLoader(template_dir),
    trim_blocks=True,
    lstrip_blocks=True)
template = env.get_template(template_file)

with open(vars_file) as f:
    vars_dict = yaml.safe_load(f)

print(template.render(vars_dict))

router bgp 100
 neighbor 10.0.0.2 remote-as 100
 neighbor 10.0.0.2 update-source lo100
 neighbor 10.0.0.3 remote-as 100
 neighbor 10.0.0.3 update-source lo100

router bgp 100
  neighbor 10.0.0.2 remote-as 100
 neighbor 10.0.0.2 update-source lo100
  neighbor 10.0.0.3 remote-as 100
 neighbor 10.0.0.3 update-source lo100
 


It will be disabled for the end of the block:

### Removing whitespace from block

Similarly, you can control whitespace removal for a block.

For this example, flags are not set in environment:

```
env = Environment(loader=FileSystemLoader(TEMPLATE_DIR))
```

Template templates/env_flags3-1.txt:

```
router bgp {{ bgp.local_as }}
 {% for ibgp in bgp.ibgp_neighbors %}
 neighbor {{ ibgp }} remote-as {{ bgp.local_as }}
 neighbor {{ ibgp }} update-source {{ bgp.loopback }}
 {% endfor %}

router bgp {{ bgp.local_as }}
 {%- for ibgp in bgp.ibgp_neighbors %}
 neighbor {{ ibgp }} remote-as {{ bgp.local_as }}
 neighbor {{ ibgp }} update-source {{ bgp.loopback }}
 {% endfor %}
 ```

Note the minus at the beginning of second block. Minus removes all whitespace characters, in this case, at the beginning of the block.

The result will be:

In [ ]:
from jinja2 import Environment, FileSystemLoader
import yaml

template_dir = "templates"
template_file = "env_flags3-1.txt"
vars_file = 'data_files/router.yml'

env = Environment(
    loader=FileSystemLoader(template_dir))
template = env.get_template(template_file)

with open(vars_file) as f:
    vars_dict = yaml.safe_load(f)

print(template.render(vars_dict))

router bgp 100
 
 neighbor 10.0.0.2 remote-as 100
 neighbor 10.0.0.2 update-source lo100
 
 neighbor 10.0.0.3 remote-as 100
 neighbor 10.0.0.3 update-source lo100
 

router bgp 100
 neighbor 10.0.0.2 remote-as 100
 neighbor 10.0.0.2 update-source lo100
 
 neighbor 10.0.0.3 remote-as 100
 neighbor 10.0.0.3 update-source lo100
 


If you add minus to the end of the block: - env_flags3-2.txt

```
router bgp {{ bgp.local_as }}
 {% for ibgp in bgp.ibgp_neighbors %}
 neighbor {{ ibgp }} remote-as {{ bgp.local_as }}
 neighbor {{ ibgp }} update-source {{ bgp.loopback }}
 {% endfor %}

router bgp {{ bgp.local_as }}
 {%- for ibgp in bgp.ibgp_neighbors %}
 neighbor {{ ibgp }} remote-as {{ bgp.local_as }}
 neighbor {{ ibgp }} update-source {{ bgp.loopback }}
 {%- endfor %}
 ```

Empty string at the end of the block will be deleted:

In [ ]:
from jinja2 import Environment, FileSystemLoader
import yaml

template_dir = "templates"
template_file = "env_flags3-2.txt"
vars_file = 'data_files/router.yml'

env = Environment(
    loader=FileSystemLoader(template_dir))
template = env.get_template(template_file)

with open(vars_file) as f:
    vars_dict = yaml.safe_load(f)

print(template.render(vars_dict))

router bgp 100
 
 neighbor 10.0.0.2 remote-as 100
 neighbor 10.0.0.2 update-source lo100
 
 neighbor 10.0.0.3 remote-as 100
 neighbor 10.0.0.3 update-source lo100
 

router bgp 100
 neighbor 10.0.0.2 remote-as 100
 neighbor 10.0.0.2 update-source lo100
 neighbor 10.0.0.3 remote-as 100
 neighbor 10.0.0.3 update-source lo100


Try to add minus at the end of expressions describing the block and look at the result: - env_flags3-3.txt

```
router bgp {{ bgp.local_as }}
 {% for ibgp in bgp.ibgp_neighbors %}
 neighbor {{ ibgp }} remote-as {{ bgp.local_as }}
 neighbor {{ ibgp }} update-source {{ bgp.loopback }}
 {% endfor %}

router bgp {{ bgp.local_as }}
 {%- for ibgp in bgp.ibgp_neighbors -%}
 neighbor {{ ibgp }} remote-as {{ bgp.local_as }}
 neighbor {{ ibgp }} update-source {{ bgp.loopback }}
 {%- endfor -%}
 ```

In [ ]:
from jinja2 import Environment, FileSystemLoader
import yaml

template_dir = "templates"
template_file = "env_flags3-3.txt"
vars_file = 'data_files/router.yml'

env = Environment(
    loader=FileSystemLoader(template_dir))
template = env.get_template(template_file)

with open(vars_file) as f:
    vars_dict = yaml.safe_load(f)

print(template.render(vars_dict))

router bgp 100
 
 neighbor 10.0.0.2 remote-as 100
 neighbor 10.0.0.2 update-source lo100
 
 neighbor 10.0.0.3 remote-as 100
 neighbor 10.0.0.3 update-source lo100
 

router bgp 100
 neighbor 10.0.0.2 remote-as 100
 neighbor 10.0.0.2 update-source lo100
 neighbor 10.0.0.3 remote-as 100
 neighbor 10.0.0.3 update-source lo100


## Variables



Variables in template are given in double curly braces:



```
hostname {{ name }}

interface Loopback0
 ip address 10.0.0.{{ id }} 255.255.255.255
```



Variable values are set based on dictionary that is passed to template.

Variable that is passed on in a dictionary may not only be a number or a string, but also for example, a list or a dictionary. Inside template, you can refer to the item by number or key.

Template example templates/variables.txt with usage of different variable variants:

```
hostname {{ name }}

interface Loopback0
 ip address 10.0.0.{{ id }} 255.255.255.255

vlan {{ vlans[0] }}

router ospf 1
 router-id 10.0.0.{{ id }}
 auto-cost reference-bandwidth 10000
 network {{ ospf.network }} area {{ ospf['area'] }}
```

And corresponding data_files/vars.yml file with variables:



```
id: 3
name: R3
vlans:
  - 10
  - 20
  - 30
ospf:
  network: 10.0.1.0 0.0.0.255
  area: 0
```

Note the use of vlans variable in template: since vlans variable is a list, you can specify which item from list we need

If a dictionary is passed (as in case of ospf variable), you can refer to dictionary objects inside template using one of the variants: ospf.network or ospf['network']

The result will be:

In [ ]:
from jinja2 import Environment, FileSystemLoader
import yaml

template_dir = "templates"
template_file = "variables.txt"
vars_file = 'data_files/vars.yml'

env = Environment(
    loader=FileSystemLoader(template_dir))
template = env.get_template(template_file)

with open(vars_file) as f:
    vars_dict = yaml.safe_load(f)

print(template.render(vars_dict))

hostname R3

interface Loopback0
 ip address 10.0.0.3 255.255.255.255

vlan 10

router ospf 1
 router-id 10.0.0.3
 auto-cost reference-bandwidth 10000
 network 10.0.1.0 0.0.0.255 area 0



## Loop for

Loop for allows you to walk through sequence of elements.

Loop for must be written inside {% %}. Furthermore, the end of the loop must be explicitly indicated:

```
{% for vlan in vlans %}
  vlan {{ vlan }}
{% endfor %}
```

Template example templates/for.txt using a loop:

```
hostname {{ name }}

interface Loopback0
 ip address 10.0.0.{{ id }} 255.255.255.255

{% for vlan, name in vlans.items() %}
vlan {{ vlan }}
 name {{ name }}
{% endfor %}

router ospf 1
 router-id 10.0.0.{{ id }}
 auto-cost reference-bandwidth 10000
 {% for networks in ospf %}
 network {{ networks.network }} area {{ networks.area }}
 {% endfor %}
 ```

File data_files/for.yml with variables:

```
id: 3
name: R3
vlans:
  10: Marketing
  20: Voice
  30: Management
ospf:
  - network: 10.0.1.0 0.0.0.255
    area: 0
  - network: 10.0.2.0 0.0.0.255
    area: 2
  - network: 10.1.1.0 0.0.0.255
    area: 0
```

In for, it is possible to go through both the list elements (for example, ospf list) and the dictionary (vlans dictionary). And similarly, through any sequence.

The result will be:

In [ ]:
from jinja2 import Environment, FileSystemLoader
import yaml

template_dir = "templates"
template_file = "for.txt"
vars_file = 'data_files/for.yml'

env = Environment(
    loader=FileSystemLoader(template_dir),
    trim_blocks=True,
    lstrip_blocks=True)
template = env.get_template(template_file)

with open(vars_file) as f:
    vars_dict = yaml.safe_load(f)

print(template.render(vars_dict))

hostname R3

interface Loopback0
 ip address 10.0.0.3 255.255.255.255

vlan 10
 name Marketing
vlan 20
 name Voice
vlan 30
 name Management

router ospf 1
 router-id 10.0.0.3
 auto-cost reference-bandwidth 10000
 network 10.0.1.0 0.0.0.255 area 0
 network 10.0.2.0 0.0.0.255 area 2
 network 10.1.1.0 0.0.0.255 area 0



## if/elif/else

if allows you to add a condition to template. For example, you can use if to add parts of template depending on the presence of variables in data dictionary.

if statement must also be within inside {% %}. End of condition must be explicitly stated:

```
{% if ospf %}
router ospf 1
 router-id 10.0.0.{{ id }}
 auto-cost reference-bandwidth 10000
{% endif %}
```

Template example templates/if.txt:

```
hostname {{ name }}

interface Loopback0
 ip address 10.0.0.{{ id }} 255.255.255.255

{% for vlan, name in vlans.items() %}
vlan {{ vlan }}
 name {{ name }}
{% endfor %}

{% if ospf %}
router ospf 1
 router-id 10.0.0.{{ id }}
 auto-cost reference-bandwidth 10000
 {% for networks in ospf %}
 network {{ networks.network }} area {{ networks.area }}
 {% endfor %}
{% endif %}
```

if ospf expression works the same way as in Python: if variable exists and is not empty, the result is True. If there is no variable or it is empty, the result is False. That is, in this template the OSPF configuration is generated only if variable ospf exists and is not empty. Configuration will be generated with two data variants.

First with data_files/if.yml that does not contain ospf variable:

```
id: 3
name: R3
vlans:
  10: Marketing
  20: Voice
  30: Management
```

The result will be:

In [ ]:
from jinja2 import Environment, FileSystemLoader
import yaml

template_dir = "templates"
template_file = "if.txt"
vars_file = 'data_files/if.yml'

env = Environment(
    loader=FileSystemLoader(template_dir),
    trim_blocks=True,
    lstrip_blocks=True)
template = env.get_template(template_file)

with open(vars_file) as f:
    vars_dict = yaml.safe_load(f)

print(template.render(vars_dict))

hostname R3

interface Loopback0
 ip address 10.0.0.3 255.255.255.255

vlan 10
 name Marketing
vlan 20
 name Voice
vlan 30
 name Management




Now a similar template but with data_files/if_ospf.yml file:

```
id: 3
name: R3
vlans:
  10: Marketing
  20: Voice
  30: Management
ospf:
  - network: 10.0.1.0 0.0.0.255
    area: 0
  - network: 10.0.2.0 0.0.0.255
    area: 2
  - network: 10.1.1.0 0.0.0.255
    area: 0
```

Now the result will be:

In [ ]:
from jinja2 import Environment, FileSystemLoader
import yaml

template_dir = "templates"
template_file = "if.txt"
vars_file = 'data_files/if_ospf.yml'

env = Environment(
    loader=FileSystemLoader(template_dir),
    trim_blocks=True,
    lstrip_blocks=True)
template = env.get_template(template_file)

with open(vars_file) as f:
    vars_dict = yaml.safe_load(f)

print(template.render(vars_dict))

hostname R3

interface Loopback0
 ip address 10.0.0.3 255.255.255.255

vlan 10
 name Marketing
vlan 20
 name Voice
vlan 30
 name Management

router ospf 1
 router-id 10.0.0.3
 auto-cost reference-bandwidth 10000
 network 10.0.1.0 0.0.0.255 area 0
 network 10.0.2.0 0.0.0.255 area 2
 network 10.1.1.0 0.0.0.255 area 0



As in Python, Jinja is allowed to make branches in condition.

Template example templates/if_vlans.txt:

```
{% for intf, params in trunks.items() %}
interface {{ intf }}
 {% if params.action == 'add' %}
 switchport trunk allowed vlan add {{ params.vlans }}
 {% elif params.action == 'delete' %}
 switchport trunk allowed vlan remove {{ params.vlans }}
 {% else %}
 switchport trunk allowed vlan {{ params.vlans }}
 {% endif %}
{% endfor %}
```

Data file data_files/if_vlans.yml:

```
trunks:
  Fa0/1:
    action: add
    vlans: 10,20
  Fa0/2:
    action: only
    vlans: 10,30
  Fa0/3:
    action: delete
    vlans: 10
```

In this example, different commands are generated depending on value of action parameter.

In template you could also use this option to refer to nested dictionaries:

```
{% for intf in trunks %}
interface {{ intf }}
 {% if trunks[intf]['action'] == 'add' %}
 switchport trunk allowed vlan add {{ trunks[intf]['vlans'] }}
 {% elif trunks[intf]['action'] == 'delete' %}
 switchport trunk allowed vlan remove {{ trunks[intf]['vlans'] }}
 {% else %}
 switchport trunk allowed vlan {{ trunks[intf]['vlans'] }}
 {% endif %}
{% endfor %}
```

This will result in the following configuration:

In [ ]:
from jinja2 import Environment, FileSystemLoader
import yaml

template_dir = "templates"
template_file = "if_vlans.txt"
vars_file = 'data_files/if_vlans.yml'

env = Environment(
    loader=FileSystemLoader(template_dir),
    trim_blocks=True,
    lstrip_blocks=True)
template = env.get_template(template_file)

with open(vars_file) as f:
    vars_dict = yaml.safe_load(f)

print(template.render(vars_dict))

interface Fa0/1
 switchport trunk allowed vlan add 10,20
interface Fa0/2
 switchport trunk allowed vlan 10,30
interface Fa0/3
 switchport trunk allowed vlan remove 10



Using if you can also filter which elements of sequence will be iterated in for loop.

Template example templates/if_for.txt with filter in for loop:

```
{% for vlan, name in vlans.items() if vlan > 15 %}
vlan {{ vlan }}
 name {{ name }}
{% endfor %}
```

Data file (data_files/if_for.yml):

```
vlans:
  10: Marketing
  20: Voice
  30: Management
```

The result will be:

In [ ]:
from jinja2 import Environment, FileSystemLoader
import yaml

template_dir = "templates"
template_file = "if_for.txt"
vars_file = 'data_files/if_for.yml'

env = Environment(
    loader=FileSystemLoader(template_dir),
    trim_blocks=True,
    lstrip_blocks=True)
template = env.get_template(template_file)

with open(vars_file) as f:
    vars_dict = yaml.safe_load(f)

print(template.render(vars_dict))

vlan 20
 name Voice
vlan 30
 name Management



## Filters

In Jinja, variables can be changed by filters. Filters are separated from variable by a vertical line (pipe |) and may contain additional arguments. In addition, several filters can be applied to variable. In this case, filters are simply written consecutively and each of them is separated by a vertical line.

Jinja supports a large number of built-in filters. We will look at only a few of them. Other filters can be found in documentation.
https://jinja.palletsprojects.com/en/3.0.x/templates/#builtin-filters

You can also easily create your own filters. We will not cover this possibility but it is well documented.
https://jinja.palletsprojects.com/en/3.0.x/api/?highlight=custom%20filter#writing-filters

### default

Filter default allows you to set default value for variable. If variable is defined, it will be displayed, if variable is not defined, the value specified in default filter will be displayed.

Template example templates/filter_default.txt:

```
router ospf 1
 auto-cost reference-bandwidth {{ ref_bw | default(10000) }}
 {% for networks in ospf %}
 network {{ networks.network }} area {{ networks.area }}
 {% endfor %}
```

If variable ref_bw is defined in dictionary, its value will be set. If there is no variable, the value of 10000 will be substituted.

Data file (data_files/filter_default.yml):

```
ospf:
  - network: 10.0.1.0 0.0.0.255
    area: 0
  - network: 10.0.2.0 0.0.0.255
    area: 2
  - network: 10.1.1.0 0.0.0.255
    area: 0
```

The result of execution:

In [ ]:
from jinja2 import Environment, FileSystemLoader
import yaml

template_dir = "templates"
template_file = "filter_default.txt"
vars_file = 'data_files/filter_default.yml'

env = Environment(
    loader=FileSystemLoader(template_dir),
    trim_blocks=True,
    lstrip_blocks=True)
template = env.get_template(template_file)

with open(vars_file) as f:
    vars_dict = yaml.safe_load(f)

print(template.render(vars_dict))

router ospf 1
 auto-cost reference-bandwidth 10000
 network 10.0.1.0 0.0.0.255 area 0
 network 10.0.2.0 0.0.0.255 area 2
 network 10.1.1.0 0.0.0.255 area 0



**By default, if variable is defined and its value is empty, it will be assumed that variable and its value exist.**

If you want default value to be set also when variable is empty (i.e., treated as False in Python), you need to specify additional parameter boolean=true.

If with the same data file the template will be changed as follows: - filter_default_boolean.txt






```
router ospf 1
 auto-cost reference-bandwidth {{ ref_bw | default(10000, boolean=true) }}
{% for networks in ospf %}
 network {{ networks.network }} area {{ networks.area }}
{% endfor %}
```

For example, if data file is: - filter_default_boolean.yml


```
ref_bw: ''
ospf:
  - network: 10.0.1.0 0.0.0.255
    area: 0
  - network: 10.0.2.0 0.0.0.255
    area: 2
  - network: 10.1.1.0 0.0.0.255
    area: 0
```

The result will be:

In [ ]:
from jinja2 import Environment, FileSystemLoader
import yaml

template_dir = "templates"
template_file = "filter_default_boolean.txt"
vars_file = 'data_files/filter_default_boolean.yml'

env = Environment(
    loader=FileSystemLoader(template_dir),
    trim_blocks=True,
    lstrip_blocks=True)
template = env.get_template(template_file)

with open(vars_file) as f:
    vars_dict = yaml.safe_load(f)

print(template.render(vars_dict))

router ospf 1
 auto-cost reference-bandwidth 10000
 network 10.0.1.0 0.0.0.255 area 0
 network 10.0.2.0 0.0.0.255 area 2
 network 10.1.1.0 0.0.0.255 area 0



Instead of default(10000, boolean=true) you can write default(10000, true)

Try editing filter_default_boolean.txt by changing default=true to default=false and see the difference.

### dictsort

Filter dictsort allows you to sort the dictionary. By default, sorting is done by keys but by changing filter parameters you can sort by values.

Filter syntax:

```
dictsort(value, case_sensitive=False, by='key')
```

After dictsort sorts the dictionary, it returns a list of tuples, not a dictionary.

Template example templates/filter_dictsort.txt using dictsort filter:

```
{% for intf, params in trunks | dictsort %}
interface {{ intf }}
 {% if params.action == 'add' %}
 switchport trunk allowed vlan add {{ params.vlans }}
 {% elif params.action == 'delete' %}
 switchport trunk allowed vlan remove {{ params.vlans }}
 {% else %}
 switchport trunk allowed vlan {{ params.vlans }}
 {% endif %}
{% endfor %}
```

Note that filter awaits a dictionary, not a list of tuples or iterator.

Data file (data_files/filter_dictsort.yml):

```
trunks:
  Fa0/2:
    action: only
    vlans: 10,30
  Fa0/3:
    action: delete
    vlans: 10
  Fa0/1:
    action: add
    vlans: 10,20
```

The result of execution will be (interfaces are ordered):

In [ ]:
from jinja2 import Environment, FileSystemLoader
import yaml

template_dir = "templates"
template_file = "filter_dictsort.txt"
vars_file = 'data_files/filter_dictsort.yml'

env = Environment(
    loader=FileSystemLoader(template_dir),
    trim_blocks=True,
    lstrip_blocks=True)
template = env.get_template(template_file)

with open(vars_file) as f:
    vars_dict = yaml.safe_load(f)

print(template.render(vars_dict))

interface Fa0/1
 switchport trunk allowed vlan add 10,20
interface Fa0/2
 switchport trunk allowed vlan 10,30
interface Fa0/3
 switchport trunk allowed vlan remove 10



### join

Filter join works just like join method in Python.

With join filter you can combine sequence of elements into a string with an optional separator between elements.

Template example templates/filter_join.txt using join filter:

```
{% for intf, params in trunks | dictsort %}
interface {{ intf }}
 {% if params.action == 'add' %}
 switchport trunk allowed vlan add {{ params.vlans | join(',') }}
 {% elif params.action == 'delete' %}
 switchport trunk allowed vlan remove {{ params.vlans | join(',') }}
 {% else %}
 switchport trunk allowed vlan {{ params.vlans | join(',') }}
 {% endif %}
{% endfor %}
```

Data file (data_files/filter_join.yml):



```
trunks:
  Fa0/1:
    action: add
    vlans:
      - 10
      - 20
  Fa0/2:
    action: only
    vlans:
      - 10
      - 30
  Fa0/3:
    action: delete
    vlans:
      - 10
```

The result of execution:

In [ ]:
from jinja2 import Environment, FileSystemLoader
import yaml

template_dir = "templates"
template_file = "filter_join.txt"
vars_file = 'data_files/filter_join.yml'

env = Environment(
    loader=FileSystemLoader(template_dir),
    trim_blocks=True,
    lstrip_blocks=True)
template = env.get_template(template_file)

with open(vars_file) as f:
    vars_dict = yaml.safe_load(f)

print(template.render(vars_dict))

interface Fa0/1
 switchport trunk allowed vlan add 10,20
interface Fa0/2
 switchport trunk allowed vlan 10,30
interface Fa0/3
 switchport trunk allowed vlan remove 10



## Tests

Besides filters, Jinja also supports tests. Tests allow variables to be tested for a certain condition.

Jinja supports a large number of built-in tests. We will look at only a few of them. The rest of tests you can find in documentation.
https://jinja.palletsprojects.com/en/3.0.x/templates/#builtin-tests

### defined

Test defined allows you to check if variable is present in the data dictionary.

Template example templates/test_defined.txt:

```
router ospf 1
{% if ref_bw is defined %}
 auto-cost reference-bandwidth {{ ref_bw }}
{% else %}
 auto-cost reference-bandwidth 10000
{% endif %}
{% for networks in ospf %}
 network {{ networks.network }} area {{ networks.area }}
{% endfor %}
```

This example is more cumbersome than default filter option, but this test may be useful if depending on whether a variable is defined or not, different commands need to be executed.

Data file (data_files/test_defined.yml):



```
ospf:
  - network: 10.0.1.0 0.0.0.255
    area: 0
  - network: 10.0.2.0 0.0.0.255
    area: 2
  - network: 10.1.1.0 0.0.0.255
    area: 0
```


The result of execution:

In [ ]:
from jinja2 import Environment, FileSystemLoader
import yaml

template_dir = "templates"
template_file = "test_defined.txt"
vars_file = 'data_files/test_defined.yml'

env = Environment(
    loader=FileSystemLoader(template_dir),
    trim_blocks=True,
    lstrip_blocks=True)
template = env.get_template(template_file)

with open(vars_file) as f:
    vars_dict = yaml.safe_load(f)

print(template.render(vars_dict))

router ospf 1
 auto-cost reference-bandwidth 10000
 network 10.0.1.0 0.0.0.255 area 0
 network 10.0.2.0 0.0.0.255 area 2
 network 10.1.1.0 0.0.0.255 area 0



### iterable

Test iterable checks whether the object is an iterator. Due to these checks, it is possible to make branches in template which will take into account the type of variable.

Template templates/test_iterable.txt (indents made to make an idea of branches more clear):

```
{% for intf, params in trunks | dictsort %}
interface {{ intf }}
 {% if params.vlans is iterable %}
   {% if params.action == 'add' %}
 switchport trunk allowed vlan add {{ params.vlans | join(',') }}
   {% elif params.action == 'delete' %}
 switchport trunk allowed vlan remove {{ params.vlans | join(',') }}
   {% else %}
 switchport trunk allowed vlan {{ params.vlans | join(',') }}
   {% endif %}
 {% else %}
   {% if params.action == 'add' %}
 switchport trunk allowed vlan add {{ params.vlans }}
   {% elif params.action == 'delete' %}
 switchport trunk allowed vlan remove {{ params.vlans }}
   {% else %}
 switchport trunk allowed vlan {{ params.vlans }}
   {% endif %}
 {% endif %}
{% endfor %}
```

Data file (data_files/test_iterable.yml):

```
trunks:
  Fa0/1:
    action: add
    vlans:
      - 10
      - 20
  Fa0/2:
    action: only
    vlans:
      - 10
      - 30
  Fa0/3:
    action: delete
    vlans: 10
```

Note the last line: vlans: 10. In this case, 10 is no longer in the list and join filter does not work. But, due to is iterable test (in this case the result will be false), in this case template goes into else branch.

The result of execution:

In [ ]:
from jinja2 import Environment, FileSystemLoader
import yaml

template_dir = "templates"
template_file = "test_iterable.txt"
vars_file = 'data_files/test_iterable.yml'

env = Environment(
    loader=FileSystemLoader(template_dir),
    trim_blocks=True,
    lstrip_blocks=True)
template = env.get_template(template_file)

with open(vars_file) as f:
    vars_dict = yaml.safe_load(f)

print(template.render(vars_dict))

interface Fa0/1
 switchport trunk allowed vlan add 10,20
interface Fa0/2
 switchport trunk allowed vlan 10,30
interface Fa0/3
 switchport trunk allowed vlan remove 10



## set

You can assign values to variables inside template. These can be new variables or there may be modified values of variables that have been passed to template. In this way you can remember a value that for example was obtained by using several filters. Then use variable name instead of repeating all filters.

Template example templates/set.txt in which set expression is used to specify shorter parameter names:

```
{% for intf, params in trunks | dictsort %}
 {% set vlans = params.vlans %}
 {% set action = params.action %}

interface {{ intf }}
 {% if vlans is iterable %}
  {% if action == 'add' %}
 switchport trunk allowed vlan add {{ vlans | join(',') }}
  {% elif action == 'delete' %}
 switchport trunk allowed vlan remove {{ vlans | join(',') }}
  {% else %}
 switchport trunk allowed vlan {{ vlans | join(',') }}
  {% endif %}
 {% else %}
  {% if action == 'add' %}
 switchport trunk allowed vlan add {{ vlans }}
  {% elif action == 'delete' %}
 switchport trunk allowed vlan remove {{ vlans }}
  {% else %}
 switchport trunk allowed vlan {{ vlans }}
  {% endif %}
 {% endif %}
{% endfor %}
```

Note the second and third lines:

```
{% set vlans = params.vlans %}
{% set action = params.action %}
```

In this way new variables are created and these new values are used. It makes template look clearer.

Data file (data_files/set.yml):

```
trunks:
  Fa0/1:
    action: add
    vlans:
      - 10
      - 20
  Fa0/2:
    action: only
    vlans:
      - 10
      - 30
  Fa0/3:
    action: delete
    vlans: 10
```

The result of execution:

In [ ]:
from jinja2 import Environment, FileSystemLoader
import yaml

template_dir = "templates"
template_file = "set.txt"
vars_file = 'data_files/set.yml'

env = Environment(
    loader=FileSystemLoader(template_dir),
    trim_blocks=True,
    lstrip_blocks=True)
template = env.get_template(template_file)

with open(vars_file) as f:
    vars_dict = yaml.safe_load(f)

print(template.render(vars_dict))


interface Fa0/1
 switchport trunk allowed vlan add 10,20

interface Fa0/2
 switchport trunk allowed vlan 10,30

interface Fa0/3
 switchport trunk allowed vlan remove 10



### include

Include expression allows you to add one template to another.

Variables that are transmitted as data must contain all data for both the master template and the one that is added through include.

Template templates/vlans.txt:

```
{% for vlan, name in vlans.items() %}
vlan {{ vlan }}
 name {{ name }}
{% endfor %}
```

Template templates/ospf.txt:

```
router ospf 1
 auto-cost reference-bandwidth 10000
{% for networks in ospf %}
 network {{ networks.network }} area {{ networks.area }}
{% endfor %}
```

Template templates/bgp.txt:

```
router bgp {{ bgp.local_as }}
{% for ibgp in bgp.ibgp_neighbors %}
 neighbor {{ ibgp }} remote-as {{ bgp.local_as }}
 neighbor {{ ibgp }} update-source {{ bgp.loopback }}
{% endfor %}
{% for ebgp in bgp.ebgp_neighbors %}
 neighbor {{ ebgp }} remote-as {{ bgp.ebgp_neighbors[ebgp] }}
{% endfor %}
```

Template templates/switch.txt uses created templates ospf and vlans:

```
{% include 'vlans.txt' %}

{% include 'ospf.txt' %}
```

Data file for configuration generation (data_files/switch.yml):

```
vlans:
  10: Marketing
  20: Voice
  30: Management
ospf:
  - network: 10.0.1.0 0.0.0.255
    area: 0
  - network: 10.0.2.0 0.0.0.255
    area: 2
  - network: 10.1.1.0 0.0.0.255
    area: 0
```

The result of script execution:

In [ ]:
from jinja2 import Environment, FileSystemLoader
import yaml

template_dir = "templates"
template_file = "switch.txt"
vars_file = 'data_files/switch.yml'

env = Environment(
    loader=FileSystemLoader(template_dir),
    trim_blocks=True,
    lstrip_blocks=True)
template = env.get_template(template_file)

with open(vars_file) as f:
    vars_dict = yaml.safe_load(f)

print(template.render(vars_dict))

vlan 10
 name Marketing
vlan 20
 name Voice
vlan 30
 name Management

router ospf 1
 auto-cost reference-bandwidth 10000
 network 10.0.1.0 0.0.0.255 area 0
 network 10.0.2.0 0.0.0.255 area 2
 network 10.1.1.0 0.0.0.255 area 0



The resulting configuration is as if lines from templates ospf.txt and vlans.txt were in switch.txt template.

Template templates/router.txt:

```
{% include 'ospf.txt' %}

{% include 'bgp.txt' %}

logging {{ log_server }}
```

In this case, in addition to include, another line in template was added to show that include expressions can be mixed with normal template.

Data file (data_files/router.yml):

```
ospf:
  - network: 10.0.1.0 0.0.0.255
    area: 0
  - network: 10.0.2.0 0.0.0.255
    area: 2
  - network: 10.1.1.0 0.0.0.255
    area: 0
bgp:
  local_as: 100
  loopback: lo100
  ibgp_neighbors:
    - 10.0.0.2
    - 10.0.0.3
  ebgp_neighbors:
    90.1.1.1: 500
    80.1.1.1: 600
log_server: 10.1.1.1
```

The result of script execution will be:



In [ ]:
from jinja2 import Environment, FileSystemLoader
import yaml

template_dir = "templates"
template_file = "router.txt"
vars_file = 'data_files/router.yml'

env = Environment(
    loader=FileSystemLoader(template_dir),
    trim_blocks=True,
    lstrip_blocks=True)
template = env.get_template(template_file)

with open(vars_file) as f:
    vars_dict = yaml.safe_load(f)

print(template.render(vars_dict))

router ospf 1
 auto-cost reference-bandwidth 10000
 network 10.0.1.0 0.0.0.255 area 0
 network 10.0.2.0 0.0.0.255 area 2
 network 10.1.1.0 0.0.0.255 area 0

router bgp 100
 neighbor 10.0.0.2 remote-as 100
 neighbor 10.0.0.2 update-source lo100
 neighbor 10.0.0.3 remote-as 100
 neighbor 10.0.0.3 update-source lo100
 neighbor 90.1.1.1 remote-as 500
 neighbor 80.1.1.1 remote-as 600

logging 10.1.1.1


Thanks to include, template templates/ospf.txt is used both in template templates/switch.txt and in template templates/router.txt, instead of repeating the same thing twice.

# Template inheritance

Template inheritance is a very powerful functionality that avoids repetition of the same in different templates.

When using inheritance, there are:

- base template - template that describes template skeleton.
this template may contain any ordinary expressions or text. In addition, special blocks are defined in this template.
- child template - template that extends base template by filling in specified blocks.
child templates can overwrite or supplement blocks defined in base template.

Example of base template templates/base_router.txt:

```
!
{% block services %}
service timestamps debug datetime msec localtime show-timezone year
service timestamps log datetime msec localtime show-timezone year
service password-encryption
service sequence-numbers
{% endblock %}
!
no ip domain lookup
!
ip ssh version 2
!
{% block ospf %}
router ospf 1
 auto-cost reference-bandwidth 10000
{% endblock %}
!
{% block bgp %}
{% endblock %}
!
{% block alias %}
{% endblock %}
!
line con 0
 logging synchronous
 history size 100
line vty 0 4
 logging synchronous
 history size 100
 transport input ssh
!
```

Note four blocks that are created in template:

```
{% block services %}
service timestamps debug datetime msec localtime show-timezone year
service timestamps log datetime msec localtime show-timezone year
service password-encryption
service sequence-numbers
{% endblock %}
!
{% block ospf %}
router ospf 1
 auto-cost reference-bandwidth 10000
{% endblock %}
!
{% block bgp %}
{% endblock %}
!
{% block alias %}
{% endblock %}
```

These are blanks for the corresponding configuration sections. A child template that uses this base template as a base can fill all or only some of the blocks.

Child template templates/hq_router.txt:

```
{% extends "base_router.txt" %}

{% block ospf %}
{{ super() }}
{% for networks in ospf %}
 network {{ networks.network }} area {{ networks.area }}
{% endfor %}
{% endblock %}

{% block alias %}
alias configure sh do sh
alias exec ospf sh run | s ^router ospf
alias exec bri show ip int bri | exc unass
alias exec id show int desc
alias exec top sh proc cpu sorted | excl 0.00%__0.00%__0.00%
alias exec c conf t
alias exec diff sh archive config differences nvram:startup-config system:running-config
alias exec desc sh int desc | ex down
{% endblock %}
```

The first line in template templates/hq_router.txt is very important:



```
{% extends "base_router.txt" %}
```

It is said that template hq_router.txt will be constructed on the basis of template base_router.txt.

Inside child template, everything happens inside blocks. Due to the blocks that have been defined in base template, child template can extend the parent template.

Note that lines described in child template outside blocks are ignored.

There are four blocks in base template: services, ospf, bgp, alias. In child template only two of them are filled: ospf and alias. That’s the convenience of inheritance. You don’t have to fill all blocks in every child template.

In this way ospf and alias blocks are used differently. In base template, ospf block already has part of configuration:

```
{% block ospf %}
router ospf 1
 auto-cost reference-bandwidth 10000
{% endblock %}
```

Therefore, child template has a choice: use this configuration and supplement it or completely rewrite everything in child template.

In this case the configuration is supplemented. That is why in child template templates/hq_router.txt the ospf block starts with expression {{ super() }}:

```
{% block ospf %}
{{ super() }}
 {% for networks in ospf %}
 network {{ networks.network }} area {{ networks.area }}
 {% endfor %}
{% endblock %}
```

{{ super() }} transfers content of this block from parent template to child template. Because of this, lines from parent are moved to child template.

Expression super doesn’t have to be at the beginning of the block. It could be anywhere in the block. Content of base template are moved to where super expression is located.

alias block simply describes the alias. And even if there were some settings in parent template, they would be substituted by content of child template.

Let’s recap the rules for working with blocks. If block is created in parent template:

- no content - in child template you can fill this block or ignore it. If block is filled, it will contain only what was written in child template (example - alias block)
- with content - in child template you can perform such actions:
    - ignore block - in this case, child template will get content from parent template (example - services block)
    -  rewrite block - then child template will contain only what it has
    - move content of the block from parent template and supplement it - then child template will contain both the content of the block from parent template and the content from child template. To pass content from parent template the expression {{ super() }} is used (example - ospf block)

Data file for template configuration generation (data_files/hq_router.yml):

```
ospf:
  - network: 10.0.1.0 0.0.0.255
    area: 0
  - network: 10.0.2.0 0.0.0.255
    area: 2
  - network: 10.1.1.0 0.0.0.255
    area: 0
```

The result will be:

In [ ]:
from jinja2 import Environment, FileSystemLoader
import yaml

template_dir = "templates"
template_file = "hq_router.txt"
vars_file = 'data_files/hq_router.yml'

env = Environment(
    loader=FileSystemLoader(template_dir),
    trim_blocks=True,
    lstrip_blocks=True)
template = env.get_template(template_file)

with open(vars_file) as f:
    vars_dict = yaml.safe_load(f)

print(template.render(vars_dict))

!
service timestamps debug datetime msec localtime show-timezone year
service timestamps log datetime msec localtime show-timezone year
service password-encryption
service sequence-numbers
!
no ip domain lookup
!
ip ssh version 2
!
router ospf 1
 auto-cost reference-bandwidth 10000

 network 10.0.1.0 0.0.0.255 area 0
 network 10.0.2.0 0.0.0.255 area 2
 network 10.1.1.0 0.0.0.255 area 0
!
!
alias configure sh do sh
alias exec ospf sh run | s ^router ospf
alias exec bri show ip int bri | exc unass
alias exec id show int desc
alias exec top sh proc cpu sorted | excl 0.00%__0.00%__0.00%
alias exec c conf t
alias exec diff sh archive config differences nvram:startup-config system:running-config
alias exec desc sh int desc | ex down
!
line con 0
 logging synchronous
 history size 100
line vty 0 4
 logging synchronous
 history size 100
 transport input ssh
!


Note that in ospf block there are commands from base template and commands from child template.